In [2]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, pipeline
import warnings
warnings.filterwarnings('ignore')


In [3]:
USE_HUGGINGFACE = True

In [4]:
MODEL_PATH = "didulantha/sms-spam-detector"

In [5]:
try:
    classifier = pipeline("text-classification", model=MODEL_PATH)
    print("✓ Model loaded successfully!\n")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("\nMake sure:")
    print("1. Model is trained and saved in ./sms-spam-detector/")
    print("2. Or update MODEL_PATH with your HuggingFace model ID")
    exit(1)

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


✓ Model loaded successfully!



In [6]:
# Example messages covering different spam types
demo_messages = [
    # Clear SPAM
    {
        "text": "CONGRATULATIONS! You've WON a FREE iPhone 15! Call 555-GIFT now to claim your prize!",
        "expected": "SPAM",
        "category": "Prize Scam"
    },
    {
        "text": "URGENT: Your bank account has been locked. Click here immediately to verify: bit.ly/notascam",
        "expected": "SPAM",
        "category": "Phishing"
    },
    {
        "text": "Limited time offer! Get 90% OFF on all products! Text STOP to unsubscribe. Reply NOW!",
        "expected": "SPAM",
        "category": "Marketing"
    },
    {
        "text": "You have been selected for a $1000 Walmart gift card. Claim here: freegiift.com",
        "expected": "SPAM",
        "category": "Gift Card Scam"
    },
    {
        "text": "XXX Hot singles in your area waiting to meet you! Click now for instant access!!!",
        "expected": "SPAM",
        "category": "Adult Content"
    },
    
    # Clear HAM (Normal messages)
    {
        "text": "Hey, are we still meeting for lunch tomorrow at 1pm? Let me know!",
        "expected": "HAM",
        "category": "Social"
    },
    {
        "text": "Your Amazon order #123-4567890 has been shipped and will arrive by Thursday.",
        "expected": "HAM",
        "category": "Legitimate Business"
    },
    {
        "text": "Can you pick up milk and eggs on your way home? Thanks!",
        "expected": "HAM",
        "category": "Family"
    },
    {
        "text": "Meeting rescheduled to 3pm. Conference room B. See you there.",
        "expected": "HAM",
        "category": "Work"
    },
    {
        "text": "Happy birthday! 🎂 Hope you have an amazing day! Let's celebrate soon!",
        "expected": "HAM",
        "category": "Personal"
    }
]


In [7]:
correct = 0
total = len(demo_messages)

for i, item in enumerate(demo_messages, 1):
    text = item["text"]
    expected = item["expected"]
    category = item["category"]
    
    # Get prediction
    result = classifier(text)[0]
    
    # Convert label (LABEL_0 = HAM, LABEL_1 = SPAM)
    prediction = "SPAM" if result['label'] == 'LABEL_1' else "HAM"
    confidence = result['score']
    
    # Check if correct
    is_correct = prediction == expected
    if is_correct:
        correct += 1
    
    # Display result
    status_emoji = "✅" if is_correct else "❌"
    pred_emoji = "🚨" if prediction == "SPAM" else "✅"
    
    print(f"{status_emoji} Test {i}: [{category}]")
    print(f"   Message: \"{text[:70]}{'...' if len(text) > 70 else ''}\"")
    print(f"   Expected: {expected} | Predicted: {pred_emoji} {prediction}")
    print(f"   Confidence: {confidence:.4f} ({confidence*100:.1f}%)")
    print()

# Summary
accuracy = correct / total
print("="*70)
print(f"📊 DEMO RESULTS: {correct}/{total} correct ({accuracy*100:.1f}% accuracy)")
print("="*70)

✅ Test 1: [Prize Scam]
   Message: "CONGRATULATIONS! You've WON a FREE iPhone 15! Call 555-GIFT now to cla..."
   Expected: SPAM | Predicted: 🚨 SPAM
   Confidence: 0.9982 (99.8%)

✅ Test 2: [Phishing]
   Message: "URGENT: Your bank account has been locked. Click here immediately to v..."
   Expected: SPAM | Predicted: 🚨 SPAM
   Confidence: 0.8401 (84.0%)

✅ Test 3: [Marketing]
   Message: "Limited time offer! Get 90% OFF on all products! Text STOP to unsubscr..."
   Expected: SPAM | Predicted: 🚨 SPAM
   Confidence: 0.9979 (99.8%)

✅ Test 4: [Gift Card Scam]
   Message: "You have been selected for a $1000 Walmart gift card. Claim here: free..."
   Expected: SPAM | Predicted: 🚨 SPAM
   Confidence: 0.9971 (99.7%)

✅ Test 5: [Adult Content]
   Message: "XXX Hot singles in your area waiting to meet you! Click now for instan..."
   Expected: SPAM | Predicted: 🚨 SPAM
   Confidence: 0.9977 (99.8%)

✅ Test 6: [Social]
   Message: "Hey, are we still meeting for lunch tomorrow at 1pm? Let me know

In [8]:
class SpamDetector:
    """Production-ready spam detector with detailed outputs"""
    
    def __init__(self, model_path):
        """Initialize the detector"""
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        print(f"\nLoading model components...")
        self.tokenizer = DistilBertTokenizer.from_pretrained(model_path)
        self.model = DistilBertForSequenceClassification.from_pretrained(model_path)
        self.model.to(self.device)
        self.model.eval()
        
        print(f"✓ Model loaded on {self.device}")
        print(f"✓ Parameters: {sum(p.numel() for p in self.model.parameters()):,}")
    
    def predict(self, text, threshold=0.5, return_details=True):
        """
        Predict if a message is spam
        
        Args:
            text: Input message
            threshold: Classification threshold (default: 0.5)
            return_details: Return detailed dict if True
        
        Returns:
            dict with prediction details or bool
        """
        # Tokenize
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=128,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        input_ids = encoding['input_ids'].to(self.device)
        attention_mask = encoding['attention_mask'].to(self.device)
        
        # Predict
        with torch.no_grad():
            outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
            probs = torch.softmax(outputs.logits, dim=1)
            spam_prob = probs[0][1].item()
            ham_prob = probs[0][0].item()
        
        is_spam = spam_prob > threshold
        
        if return_details:
            return {
                'text': text,
                'is_spam': is_spam,
                'label': 'SPAM' if is_spam else 'HAM',
                'spam_probability': spam_prob,
                'ham_probability': ham_prob,
                'confidence': spam_prob if is_spam else ham_prob,
                'threshold': threshold
            }
        else:
            return is_spam
    
    def predict_batch(self, texts, threshold=0.5):
        """Predict for multiple messages"""
        return [self.predict(text, threshold) for text in texts]
    
    def explain_prediction(self, text):
        """Get detailed explanation of prediction"""
        result = self.predict(text)
        
        print(f"\n{'='*70}")
        print("PREDICTION EXPLANATION")
        print(f"{'='*70}")
        print(f"\n📧 Message:")
        print(f"   \"{text}\"")
        print(f"\n🎯 Prediction: {result['label']}")
        print(f"   Spam Probability: {result['spam_probability']:.4f} ({result['spam_probability']*100:.1f}%)")
        print(f"   Ham Probability:  {result['ham_probability']:.4f} ({result['ham_probability']*100:.1f}%)")
        print(f"   Confidence: {result['confidence']:.4f} ({result['confidence']*100:.1f}%)")
        print(f"   Decision: {'🚨 SPAM' if result['is_spam'] else '✅ LEGITIMATE'}")
        
        # Risk assessment
        if result['spam_probability'] > 0.9:
            risk = "🔴 VERY HIGH - Almost certainly spam"
        elif result['spam_probability'] > 0.7:
            risk = "🟠 HIGH - Likely spam"
        elif result['spam_probability'] > 0.5:
            risk = "🟡 MEDIUM - Possibly spam"
        elif result['spam_probability'] > 0.3:
            risk = "🟢 LOW - Probably legitimate"
        else:
            risk = "⚪ VERY LOW - Almost certainly legitimate"
        
        print(f"\n⚠️  Risk Level: {risk}")
        print(f"{'='*70}")
        
        return result


In [9]:
# Initialize advanced detector
detector = SpamDetector(MODEL_PATH)


Loading model components...
✓ Model loaded on cuda
✓ Parameters: 66,955,010


In [10]:
test_cases = [
    "WINNER! Claim your FREE $1000 now! Limited time only!",
    "Hey, want to grab coffee this weekend?",
    "Your package delivery has been delayed. Track: amzn.to/track123"
]

for text in test_cases:
    detector.explain_prediction(text)
    print()


PREDICTION EXPLANATION

📧 Message:
   "WINNER! Claim your FREE $1000 now! Limited time only!"

🎯 Prediction: SPAM
   Spam Probability: 0.9978 (99.8%)
   Ham Probability:  0.0022 (0.2%)
   Confidence: 0.9978 (99.8%)
   Decision: 🚨 SPAM

⚠️  Risk Level: 🔴 VERY HIGH - Almost certainly spam


PREDICTION EXPLANATION

📧 Message:
   "Hey, want to grab coffee this weekend?"

🎯 Prediction: HAM
   Spam Probability: 0.0008 (0.1%)
   Ham Probability:  0.9992 (99.9%)
   Confidence: 0.9992 (99.9%)
   Decision: ✅ LEGITIMATE

⚠️  Risk Level: ⚪ VERY LOW - Almost certainly legitimate


PREDICTION EXPLANATION

📧 Message:
   "Your package delivery has been delayed. Track: amzn.to/track123"

🎯 Prediction: SPAM
   Spam Probability: 0.9976 (99.8%)
   Ham Probability:  0.0024 (0.2%)
   Confidence: 0.9976 (99.8%)
   Decision: 🚨 SPAM

⚠️  Risk Level: 🔴 VERY HIGH - Almost certainly spam



In [11]:
# INTERACTIVE MODE
def interactive_mode():
    """Interactive testing"""
    print("\n💬 Enter messages to test (type 'quit' to exit)")
    print("   Or press Enter for random examples\n")
    
    example_messages = [
        "Free iPhone! Click now!",
        "Meeting at 3pm tomorrow",
        "URGENT: Account suspended",
        "Can you call me later?",
        "Congratulations, you won!",
        "Thanks for the help yesterday"
    ]
    
    while True:
        user_input = input("📱 Enter message (or press Enter for example): ").strip()
        
        if user_input.lower() in ['quit', 'exit', 'q']:
            print("\n👋 Thanks for testing!")
            break
        
        # Use example if no input
        if not user_input:
            import random
            user_input = random.choice(example_messages)
            print(f"   Using example: \"{user_input}\"")
        
        # Get prediction
        detector.explain_prediction(user_input)
        print()

In [13]:
try:
    choice = input().lower().strip()
    if choice == 'y':
        interactive_mode()
except KeyboardInterrupt:
    print("\n\n👋 Exiting...")

 h


In [14]:
batch_messages = [
    "Win a free vacation to Hawaii!",
    "Your order has been confirmed",
    "Click here for instant loan approval",
    "See you at the party tonight!",
    "URGENT: Verify your account now"
]

print(f"\nProcessing {len(batch_messages)} messages...\n")

batch_results = detector.predict_batch(batch_messages)

for msg, result in zip(batch_messages, batch_results):
    emoji = "🚨" if result['is_spam'] else "✅"
    print(f"{emoji} {result['label']}: \"{msg}\"")
    print(f"   Confidence: {result['confidence']:.4f}\n")


Processing 5 messages...

🚨 SPAM: "Win a free vacation to Hawaii!"
   Confidence: 0.6485

✅ HAM: "Your order has been confirmed"
   Confidence: 0.9980

✅ HAM: "Click here for instant loan approval"
   Confidence: 0.6938

✅ HAM: "See you at the party tonight!"
   Confidence: 0.9986

✅ HAM: "URGENT: Verify your account now"
   Confidence: 0.9988



In [17]:
import time

# Single inference timing
text = "Test message for performance measurement"
start = time.time()
for _ in range(100):
    _ = detector.predict(text, return_details=False)
end = time.time()

avg_time = (end - start) / 100 * 1000  # Convert to ms

print(f"\n  Performance:")
print(f"   Average inference time: {avg_time:.2f}ms")
print(f"   Throughput: {1000/avg_time:.0f} messages/second")
print(f"   Device: {detector.device}")





  Performance:
   Average inference time: 5.90ms
   Throughput: 170 messages/second
   Device: cuda
